In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/My Drive/CS224U/OSD_words.txt', 'r') as f:
    words = f.read().splitlines()

In [ ]:
for i in range(len(words)):
    words[i] = words[i].replace(' ', '-')
    words[i].lower()

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(words, columns=['word_id'])

In [ ]:
len(df)

19594

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def scrape_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    class_elem = soup.find('div', class_='term featured')
    a_tag = class_elem.find('a') if class_elem else None
    word = a_tag.text if a_tag else None

    blockquotes = soup.find_all('blockquote', class_='sentence')
    usages = []
    if blockquotes:
        for blockquote in blockquotes:
            sentence = blockquote.text.strip()
            tokens = word_tokenize(sentence)
            usages.append(tokens)
    # usages = [blockquote.text for blockquote in blockquotes] if blockquotes else None


    def_class = soup.find(class_="definitions")
    li_tag = def_class.find('li') if def_class else None
    defin = li_tag.text if li_tag else None
    defin = defin.split('.', 1)[0] if defin else None
    
    return (word, defin, usages)


In [ ]:
df['word'] = None
df['definitions'] = None
df['usages'] = None

In [ ]:
df.head(10)

,word_id,word,definitions,usages
0,0-day,None,None,None
1,1,None,None,None
2,10,None,None,None
3,100%-all-beef-thermometer,None,None,None
4,101,None,None,None
5,"101-scoop,-the",None,None,None
6,10-20,None,None,None
7,10-2-2-10,None,None,None
8,10:30,None,None,None
9,10-4,None,None,None


In [ ]:
import itertools
for index, row in df.iterrows():
    word_id = row['word_id']
    url = f'http://onlineslangdictionary.com/meaning-definition-of/{word_id}'
      
    # Call the helper function with the constructed URL
    word, defin, usages = scrape_page(url)
    
    # Update the corresponding row in the DataFrame with the scraped data
    df.at[index, 'word'] = word
    df.at[index, 'usages'] = usages
    df.at[index, 'definitions'] = defin
    print(index)


In [ ]:
copy = df.copy()
copy.head(10)

,word_id,word,definitions,usages
0,0-day,0 day,commercial software available for piracy on th...,"[[i, have, vb, 0day, .]]"
1,1,1,"""goodbye""","[[i, 'll, talk, to, you, later, ., 1, .], [see..."
2,10,10,"A person who's really hot, even more than others","[[she, was, only, the, 10, in, the, crowd, .]]"
3,100%-all-beef-thermometer,None,None,[]
4,101,101,a beginner's course,"[[my, boyfriend, needs, to, re-take, sex, 101,..."
5,"101-scoop,-the",None,"the real information or the proof, instruction...","[[i, got, the, 101, scoop, about, the, crime, ..."
6,10-20,10-20,"""What is your 20?"" i","[[citation, from, austin, powers, in, goldmemb..."
7,10-2-2-10,None,an unattractive female met while drinking alco...,[]
8,10:30,None,None,[]
9,10-4,None,"""message understood","[[citation, from, austin, powers, in, goldmemb..."


In [ ]:
copy = copy[copy['usages'].apply(lambda x: len(x) >0)]
copy.head(10)

,word_id,word,definitions,usages
0,0-day,0 day,commercial software available for piracy on th...,"[[i, have, vb, 0day, .]]"
1,1,1,"""goodbye""","[[i, 'll, talk, to, you, later, ., 1, .], [see..."
2,10,10,"A person who's really hot, even more than others","[[she, was, only, the, 10, in, the, crowd, .]]"
4,101,101,a beginner's course,"[[my, boyfriend, needs, to, re-take, sex, 101,..."
5,"101-scoop,-the",None,"the real information or the proof, instruction...","[[i, got, the, 101, scoop, about, the, crime, ..."
6,10-20,10-20,"""What is your 20?"" i","[[citation, from, austin, powers, in, goldmemb..."
9,10-4,None,"""message understood","[[citation, from, austin, powers, in, goldmemb..."
10,1080,1080,"in sports, the turning around of 3 times while...","[[citation, from, ``, skateboarding, will, be,..."
12,11,11,an extremely attractive person - more attracti...,"[[dude, ,, you, have, no, chance, with, her, ...."
14,110-percent,110 percent,"110%- indicating a statement is true, beyond a...","[[sports, team, coach, :, i, want, each, and, ..."


In [ ]:
copy = copy.dropna(subset=['usages'])

In [ ]:
for index, row in copy.iterrows():
    for sublist_index, sublist in enumerate(row['usages']):
        lowercase_sublist = [word.lower() for word in sublist]
        copy.at[index, 'usages'][sublist_index] = lowercase_sublist

In [ ]:
copy['word'] = copy['word'].astype(str).str.lower()

In [ ]:
final = copy.copy()

In [ ]:
# Iterate over each row in the DataFrame
for index, row in final.iterrows():
    if str(row['word']).lower() == 'none':  # Check if 'word' column value is 'none' (case-insensitive)
        word_id = row['word_id']  # Get the value from 'word_id' column
        modified_word = word_id.replace("-", " ")  # Replace dashes with space
        final.at[index, 'word'] = modified_word  # Assign modified value to 'word' column


In [ ]:
final['word'] = final['word'].apply(nltk.word_tokenize)

In [ ]:
final['word_id'] = final['word_id'].apply(nltk.word_tokenize)

In [ ]:
for index, row in final.iterrows():
    word_list = row['word'] + row['word_id']  # Combine words from 'word' and 'word_id' columns
    row['usages'] = [sublist for sublist in row['usages'] if any(word in sublist for word in word_list)]

final.dropna(subset=['usages'], inplace=True)  # Remove rows with empty 'usage' column

In [ ]:
final = final[final['usages'].apply(lambda x: len(x) >0)]

In [ ]:
final['tags'] = None

In [ ]:
final.head(10)

,word_id,word,definitions,usages,tags
1,[1],[1],"""goodbye""","[[i, 'll, talk, to, you, later, ., 1, .], [see...",None
2,[10],[10],"A person who's really hot, even more than others","[[she, was, only, the, 10, in, the, crowd, .]]",None
4,[101],[101],a beginner's course,"[[my, boyfriend, needs, to, re-take, sex, 101,...",None
5,"[101-scoop, ,, -the]","[101, scoop, ,, the]","the real information or the proof, instruction...","[[i, got, the, 101, scoop, about, the, crime, ...",None
9,[10-4],"[10, 4]","""message understood","[[i, was, walking, up, to, this, girl, to, tal...",None
12,[11],[11],an extremely attractive person - more attracti...,"[[dude, ,, you, have, no, chance, with, her, ....",None
14,[110-percent],"[110, percent]","110%- indicating a statement is true, beyond a...","[[sports, team, coach, :, i, want, each, and, ...",None
15,[1337],[1337],"""elite""","[[man, ,, that, 's, one, 1337, computer, !], [...",None
16,[133t],[133t],"""elite","[[that, a, 133t, bike, .], [that, kid, got, 13...",None
17,[13th-step],"[13th, step]",To have sex with a fellow addict in a 12-step ...,"[[my, sponsor, said, that, a, proper, 13th, st...",None


In [ ]:
def generate_combinations(words):
    combinations_list = []
    for i in range(len(words)):
        for j in range(i + 1, len(words) + 1):
            combinations_list.append('-'.join(words[i:j]))
    return combinations_list

In [ ]:
for index, row in final.iterrows():
    final.at[index, 'tags'] = []

    slang = generate_combinations(row['word'])

    # slang = row['word'] + row['word_id']
    usages = row['usages']
    
    sublist_index = 0
    while sublist_index < len(usages):
        sublist = usages[sublist_index]
        sub_tags = []
        i = 0

        # slang = ['a-bit-much', 'a', 'bit', 'much']
        # sublist = ['he', 'wanted', 'a', 'machete', 'for', 'his', 'birthday', '.', 'i', 'thought', 'it', 'was', 'a', 'bit', 'much', '.']
        while i < len(sublist):
            if sublist[i] in slang: 
                sub_tags.append('B')

                # Check for consecutive matching words in column1
                while i + 1 < len(sublist) and sublist[i + 1] in slang:
                    sub_tags.append('I')
                    i += 1
            else:
                sub_tags.append('O')
            
            i += 1

            
        
        final.at[index, 'tags'].append(sub_tags)

        sublist_index += 1

In [ ]:
final.head(60)

,word_id,word,definitions,usages,tags
1,[1],[1],"""goodbye""","[[i, 'll, talk, to, you, later, ., 1, .], [see...","[[O, O, O, O, O, O, O, B, O], [O, O, O, O, B, ..."
2,[10],[10],"A person who's really hot, even more than others","[[she, was, only, the, 10, in, the, crowd, .]]","[[O, O, O, O, B, O, O, O, O]]"
4,[101],[101],a beginner's course,"[[my, boyfriend, needs, to, re-take, sex, 101,...","[[O, O, O, O, O, O, B, O]]"
5,"[101-scoop, ,, -the]","[101, scoop, ,, the]","the real information or the proof, instruction...","[[i, got, the, 101, scoop, about, the, crime, ...","[[O, O, B, I, I, O, B, O, O, O, O, O, O, B, O,..."
9,[10-4],"[10, 4]","""message understood","[[i, was, walking, up, to, this, girl, to, tal...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,..."
12,[11],[11],an extremely attractive person - more attracti...,"[[dude, ,, you, have, no, chance, with, her, ....","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B,..."
14,[110-percent],"[110, percent]","110%- indicating a statement is true, beyond a...","[[sports, team, coach, :, i, want, each, and, ...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,..."
15,[1337],[1337],"""elite""","[[man, ,, that, 's, one, 1337, computer, !], [...","[[O, O, O, O, O, B, O, O], [O, O, O, O, O, B, O]]"
16,[133t],[133t],"""elite","[[that, a, 133t, bike, .], [that, kid, got, 13...","[[O, O, B, O, O], [O, O, O, B, O, O, O, O, O]]"
17,[13th-step],"[13th, step]",To have sex with a fellow addict in a 12-step ...,"[[my, sponsor, said, that, a, proper, 13th, st...","[[O, O, O, O, O, O, B, I, O, O, O, O, O, O, O,..."


In [ ]:
final.to_csv('/content/drive/MyDrive/temp_file.csv', index=False)

In [ ]:
len(final)

10394

In [ ]:
# url = 'http://onlineslangdictionary.com/meaning-definition-of/101-scoop,-the'
# word, definition, example = scrape_page(url)
# print('Word:', word)
# print('Definition:', definition)
# print('Example:', example)

Word: None
Definition: the real information or the proof, instruction, the know how, where, when, and why about something or someone
Example: [['I', 'got', 'the', '101', 'scoop', 'about', 'the', 'crime', '(', 'or', 'product', 'or', 'job', ',', 'etc', '.', ')']]


In [ ]:
# # List of URLs you want to scrape
# urls = ['http://onlineslangdictionary.com/meaning-definition-of/word1', 
#         'http://onlineslangdictionary.com/meaning-definition-of/word2', 
#         'http://onlineslangdictionary.com/meaning-definition-of/word3', 
#         # Add more URLs as needed
#         ]

# for url in urls:
#     word, definition, example = scrape_page(url)
#     print('Word:', word)
#     print('Definition:', definition)
#     print('Example:', example)

#     time.sleep(1)